# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f3894cccf10>
├── 'a' --> tensor([[ 1.2517, -1.2640,  0.3041],
│                   [ 1.9151,  2.1916,  1.6329]])
└── 'x' --> <FastTreeValue 0x7f3894ccc580>
    └── 'c' --> tensor([[-0.9170, -1.6135,  1.8149,  0.2497],
                        [-0.2739, -1.7284,  0.8622, -1.7570],
                        [-0.6568,  1.1431, -1.5395, -1.8568]])

In [4]:
t.a

tensor([[ 1.2517, -1.2640,  0.3041],
        [ 1.9151,  2.1916,  1.6329]])

In [5]:
%timeit t.a

72.4 ns ± 0.595 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f3894cccf10>
├── 'a' --> tensor([[-0.4662, -1.5288, -0.8120],
│                   [-0.4578,  0.2335, -0.7567]])
└── 'x' --> <FastTreeValue 0x7f3894ccc580>
    └── 'c' --> tensor([[-0.9170, -1.6135,  1.8149,  0.2497],
                        [-0.2739, -1.7284,  0.8622, -1.7570],
                        [-0.6568,  1.1431, -1.5395, -1.8568]])

In [7]:
%timeit t.a = new_value

78.1 ns ± 1.55 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[ 1.2517, -1.2640,  0.3041],
               [ 1.9151,  2.1916,  1.6329]]),
    x: Batch(
           c: tensor([[-0.9170, -1.6135,  1.8149,  0.2497],
                      [-0.2739, -1.7284,  0.8622, -1.7570],
                      [-0.6568,  1.1431, -1.5395, -1.8568]]),
       ),
)

In [10]:
b.a

tensor([[ 1.2517, -1.2640,  0.3041],
        [ 1.9151,  2.1916,  1.6329]])

In [11]:
%timeit b.a

69.9 ns ± 0.932 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[ 0.7952,  0.3910,  2.4784],
               [ 0.0888, -0.2779, -1.2903]]),
    x: Batch(
           c: tensor([[-0.9170, -1.6135,  1.8149,  0.2497],
                      [-0.2739, -1.7284,  0.8622, -1.7570],
                      [-0.6568,  1.1431, -1.5395, -1.8568]]),
       ),
)

In [13]:
%timeit b.a = new_value

633 ns ± 4.29 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

999 ns ± 15 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

13.5 µs ± 210 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

308 µs ± 7.11 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

281 µs ± 9.93 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f37def6fb50>
├── 'a' --> tensor([[[ 1.2517, -1.2640,  0.3041],
│                    [ 1.9151,  2.1916,  1.6329]],
│           
│                   [[ 1.2517, -1.2640,  0.3041],
│                    [ 1.9151,  2.1916,  1.6329]],
│           
│                   [[ 1.2517, -1.2640,  0.3041],
│                    [ 1.9151,  2.1916,  1.6329]],
│           
│                   [[ 1.2517, -1.2640,  0.3041],
│                    [ 1.9151,  2.1916,  1.6329]],
│           
│                   [[ 1.2517, -1.2640,  0.3041],
│                    [ 1.9151,  2.1916,  1.6329]],
│           
│                   [[ 1.2517, -1.2640,  0.3041],
│                    [ 1.9151,  2.1916,  1.6329]],
│           
│                   [[ 1.2517, -1.2640,  0.3041],
│                    [ 1.9151,  2.1916,  1.6329]],
│           
│                   [[ 1.2517, -1.2640,  0.3041],
│                    [ 1.9151,  2.1916,  1.6329]]])
└── 'x' --> <FastTreeValue 0x7f37def6fbe0>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

46.5 µs ± 923 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f3894c579a0>
├── 'a' --> tensor([[ 1.2517, -1.2640,  0.3041],
│                   [ 1.9151,  2.1916,  1.6329],
│                   [ 1.2517, -1.2640,  0.3041],
│                   [ 1.9151,  2.1916,  1.6329],
│                   [ 1.2517, -1.2640,  0.3041],
│                   [ 1.9151,  2.1916,  1.6329],
│                   [ 1.2517, -1.2640,  0.3041],
│                   [ 1.9151,  2.1916,  1.6329],
│                   [ 1.2517, -1.2640,  0.3041],
│                   [ 1.9151,  2.1916,  1.6329],
│                   [ 1.2517, -1.2640,  0.3041],
│                   [ 1.9151,  2.1916,  1.6329],
│                   [ 1.2517, -1.2640,  0.3041],
│                   [ 1.9151,  2.1916,  1.6329],
│                   [ 1.2517, -1.2640,  0.3041],
│                   [ 1.9151,  2.1916,  1.6329]])
└── 'x' --> <FastTreeValue 0x7f37def1ff40>
    └── 'c' --> tensor([[-0.9170, -1.6135,  1.8149,  0.2497],
                        [-0.2739, -1.7284,  0.8622, -1.7570],
                 

In [23]:
%timeit t_cat(trees)

43.2 µs ± 1.95 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

84 µs ± 963 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    a: tensor([[[ 1.2517, -1.2640,  0.3041],
                [ 1.9151,  2.1916,  1.6329]],
       
               [[ 1.2517, -1.2640,  0.3041],
                [ 1.9151,  2.1916,  1.6329]],
       
               [[ 1.2517, -1.2640,  0.3041],
                [ 1.9151,  2.1916,  1.6329]],
       
               [[ 1.2517, -1.2640,  0.3041],
                [ 1.9151,  2.1916,  1.6329]],
       
               [[ 1.2517, -1.2640,  0.3041],
                [ 1.9151,  2.1916,  1.6329]],
       
               [[ 1.2517, -1.2640,  0.3041],
                [ 1.9151,  2.1916,  1.6329]],
       
               [[ 1.2517, -1.2640,  0.3041],
                [ 1.9151,  2.1916,  1.6329]],
       
               [[ 1.2517, -1.2640,  0.3041],
                [ 1.9151,  2.1916,  1.6329]]]),
    x: Batch(
           c: tensor([[[-0.9170, -1.6135,  1.8149,  0.2497],
                       [-0.2739, -1.7284,  0.8622, -1.7570],
                       [-0.6568,  1.1431, -1.5395, -1.8568]],
         

In [26]:
%timeit Batch.stack(batches)

104 µs ± 1.36 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    a: tensor([[ 1.2517, -1.2640,  0.3041],
               [ 1.9151,  2.1916,  1.6329],
               [ 1.2517, -1.2640,  0.3041],
               [ 1.9151,  2.1916,  1.6329],
               [ 1.2517, -1.2640,  0.3041],
               [ 1.9151,  2.1916,  1.6329],
               [ 1.2517, -1.2640,  0.3041],
               [ 1.9151,  2.1916,  1.6329],
               [ 1.2517, -1.2640,  0.3041],
               [ 1.9151,  2.1916,  1.6329],
               [ 1.2517, -1.2640,  0.3041],
               [ 1.9151,  2.1916,  1.6329],
               [ 1.2517, -1.2640,  0.3041],
               [ 1.9151,  2.1916,  1.6329],
               [ 1.2517, -1.2640,  0.3041],
               [ 1.9151,  2.1916,  1.6329]]),
    x: Batch(
           c: tensor([[-0.9170, -1.6135,  1.8149,  0.2497],
                      [-0.2739, -1.7284,  0.8622, -1.7570],
                      [-0.6568,  1.1431, -1.5395, -1.8568],
                      [-0.9170, -1.6135,  1.8149,  0.2497],
                      [-0.2739, -

In [28]:
%timeit Batch.cat(batches)

184 µs ± 3.53 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

618 µs ± 17 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
